In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

In [2]:
from main import pipeline
from core.io.loader import *
from features.collaboration import *
from features.papers_graph import *
from features.authors import *

In [3]:
papers_extractor = ExtractGraphPapersFeatures()

In [42]:
train = get_data_with_node_information('train', data_dir='../data/')

In [5]:
papers_extractor.fit(train)

In [15]:
papers_graph = papers_extractor.graph

In [17]:
nodes_mapping = papers_extractor.nodes_mapping

In [24]:
import time

In [29]:
st = time.time()
adamaic_adar = train.apply(
    lambda x: adamic_adar_index(x.source_id, x.target_id, papers_graph, nodes_mapping),
    axis='columns'
)
print(time.time() - st)

80.73672699928284


In [50]:
def extract_adamic_adar_index(source_target, graph: igraph.Graph, nodes_mapping):
    source_id, target_id = tuple(map(int, source_target.split(',')))
    source_neighbors = set(graph.neighbors(nodes_mapping[source_id]))
    target_neighbors = set(graph.neighbors(nodes_mapping[target_id]))
    metric = 0.0
    for neighbor in source_neighbors.intersection(target_neighbors):
        metric += 1 / np.log(len(graph.neighbors(neighbor)))
    return metric


In [51]:
# As pandas is column based, doing this trick divide execution time by 2
concatenated_ids = train.source_id.astype('str') + ',' + train.target_id.astype('str')
adamaic_adar2 = concatenated_ids.apply(lambda x: extract_adamic_adar_index(x, papers_graph, nodes_mapping))

In [ ]:
train.drop()

In [52]:
train['adamic_adar2'] = adamaic_adar2

In [47]:
source_id = nodes_mapping[9707075]
target_id = nodes_mapping[9604178]

source_neighbors = papers_graph.neighbors(source_id)
target_neighbors = papers_graph.neighbors(target_id)

metric = 0
for neighbor in source_neighbors:
    if neighbor in target_neighbors:
        metric += 1/np.log(len(papers_graph.neighbors(neighbor)))
metric

In [53]:
train[['source_id', 'target_id', 'adamic_adar2']]

,source_id,target_id,adamic_adar2
0,9510123,9502114,0.513898
1,9707075,9604178,4.320366
2,9312155,9506142,0.000000
3,9911255,302165,0.000000
4,9701033,209076,0.000000
5,9710020,9709228,3.175030
6,9901042,9510135,2.468741
7,209146,9502077,0.000000
8,9705079,9702201,0.942862
9,3016,9207067,0.000000


In [57]:
graph = papers_graph

In [102]:
def neighborhood_based_metrics(source_id: pd.Series, target_id: pd.Series, graph:igraph.Graph, nodes_mapping ):
    metrics = {
            'adamic_adar': [],
            'common_neighbors': [],
            'jaccard_coefficient': [],
            'preferential_attachment': []
    }
    for source_id, target_id in zip(source_id.values, target_id.values):
        source_neighbors = set(graph.neighbors(nodes_mapping[source_id]))
        target_neighbors = set(graph.neighbors(nodes_mapping[target_id]))
        neighbors_intersection = source_neighbors.intersection(target_neighbors)
        neighbors_union = source_neighbors.union(target_neighbors)
        

        # Extract adamic_adar
        adamic_adar =  0.0
        for neighbor in neighbors_intersection:
            adamic_adar += 1 / np.log(len(graph.neighbors(neighbor)))
        metrics['adamic_adar'].append(adamic_adar)
        # common_neighbors
        metrics['common_neighbors'].append(len(neighbors_intersection))

        # Jaccard coefficient
        if len(neighbors_union) > 0:
            metrics['jaccard_coefficient'].append(len(neighbors_intersection) / len(neighbors_union))
        else:
            metrics['jaccard_coefficient'].append(np.inf)

        # Preferential attachment
        metrics['preferential_attachment'].append(len(source_neighbors) * len(target_neighbors))
    return metrics

In [103]:
st = time.time()
metrics = neighborhood_based_metrics(train.source_id, train.target_id, papers_graph, nodes_mapping)
print(time.time() - st )

metrics = pd.DataFrame(metrics, index=train.index)

train = pd.concat([train, metrics], axis='columns')

54.65935516357422


###### test

In [43]:
train = papers_extractor.transform(train)


KeyboardInterrupt: 

### Create neighborhood features for authors

In [6]:
from features.authors import *

In [44]:
train = ExtractAuthorsList().transform(train)

In [45]:
collaboration_graph_features = CollaborationGraphFeatures()

In [46]:
collaboration_graph_features.fit(train)
train = collaboration_graph_features.transform(train)

### Extract neihborhood features for authors metrics

In [19]:
graph = collaboration_graph_features.collaboration_graph

In [40]:
authors_neighborhood_metrics = [
    'authors_adamic_adar','authors_common_neighbors','authors_jaccard_coefficient','authors_preferential_attachment'
    ]

train = train.drop(columns=['max_authors_adamic_adar','max_authors_common_neighbors', 'max_authors_jaccard_coefficient', 'max_authors_preferential_attachment'])

for neighborhood_metric in authors_neighborhood_metrics:
    train['max_' + neighborhood_metric] = train[neighborhood_metric].apply(
        lambda x: np.max(x) if isinstance(x, list) and len(x) >0 else None
    )